[실 구매 영수증 기반 온라인 쇼핑몰 연령별 매출 데이터 - KADX 농식품 빅데이터 거래소](https://kadx.co.kr/opmk/frn/pmumkproductDetail/PMU_9334c816-5826-4d0e-a8f4-0b9eb47f6f4d/5#!)

## 실 구매 영수증 기반 온라인 쇼핑몰 연령별 매출 데이터 
* 주력 상품 카테고리 파악: 어떤 상품 카테고리가 가장 높은 매출을 기록하는지 파악하여 마케팅 전략 수립에 활용할 수 있습니다.
* 고객 세분화: RFM 분석을 통해 고객을 세분화하여 맞춤형 마케팅 전략을 수립할 수 있습니다.
* 구매 패턴 분석: 주차별, 연령별, 제품 카테고리별 구매 패턴을 분석하여 프로모션 시행 시점이나 재고 관리 전략 수립에 활용할 수 있습니다.
* 매출 변동 분석: 주차별, 계절별 매출 변동을 분석하여 매출 예측 모델을 개발하고, 사업 계획 수립에 활용할 수 있습니다.

In [ ]:
# !pip install -Uq koreanize-matplotlib

In [ ]:
import pandas as pd
import numpy as np
import koreanize_matplotlib
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
df = pd.read_excel("data/실 구매 영수증 기반 온라인 쇼핑몰 연령별 매출 데이터.xlsx")
df.shape

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# 중복데이터 확인
df[df.duplicated()]

In [ ]:
# 결측치 확인
df.isnull().sum()

In [ ]:
# 수치형 변수의 기술 통계 확인
df.describe()

In [ ]:
# 범주형 변수의 기술 통계 확인
df.describe(include="object")

In [ ]:
# 전체 변수의 고유값(유일값)
df.nunique()

## 분류별 데이터 빈도 및 구매금액 합계

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 3))
df['대분류'].value_counts().sort_values(
    ascending=True).plot(kind='barh', title='대분류 데이터 빈도', ax=axes[0]);
df.groupby('대분류')['구매금액'].sum().sort_values(
    ascending=True).plot(kind='barh', title='대분류 구매금액 합계', ax=axes[1]);

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(24, 15))
df['중분류'].value_counts().plot(kind='bar', rot=60, ax=axes[0], title='중분류 데이터 빈도');
df.groupby('중분류')['구매금액'].sum().sort_values(
    ascending=False).plot(kind='bar', title='중분류 구매금액 합계', rot=60, ax=axes[1]);

In [ ]:
df['구매상품명'].value_counts().head(10)

In [ ]:
df.groupby('구매상품명')['구매금액'].sum().nlargest(10)

## 구매 분류별 수량 및 금액 분석

In [ ]:
# 구매 대분류별 판매 성과 분석
channel_sales_summary = df.groupby('대분류')[['구매수량', '구매금액']].agg(['sum', 'mean', 'count']).reset_index()
channel_sales_summary.style.background_gradient()

In [ ]:
# 대분류별 매출 비율
total_sales = df['구매금액'].sum()
channel_sales_summary[('매출 비율', 'percent')] = (channel_sales_summary[('구매금액', 'sum')] / total_sales) * 100
channel_sales_summary.style.background_gradient()

## 주요 구매 상품명

In [ ]:
# 대분류별 주요 구매 상품 분석
top_products_by_channel = df.groupby('대분류')['구매상품명'].apply(lambda x: ', '.join(x.value_counts().index[:10]))
for i, tp in enumerate(top_products_by_channel):
    print("-" * 100)
    print(top_products_by_channel.index[i], "===>")
    print(tp)

In [ ]:
# 구매자연령 주요 구매 상품 분석
top_products_by_channel = df.groupby('구매자연령')['구매상품명'].apply(lambda x: ', '.join(x.value_counts().index[:10]))
for i, tp in enumerate(top_products_by_channel):
    print("-" * 100)
    print(top_products_by_channel.index[i], "===>")
    print(tp)

## 주차별 구매 

In [ ]:
# 주차별 구매 패턴 분석
weekly_sales_pattern = df.groupby('구매주차')['구매금액'].sum().reset_index()
weekly_sales_pattern.plot(figsize=(10, 3))

In [ ]:
df['구매일'] = pd.to_datetime(df['구매주차일자'].str.split('~').str[0].str.strip())
daily_sales = df.groupby('구매일')['구매금액'].sum().sort_index()
daily_sales.plot(figsize=(10, 3))

In [ ]:
age_sales = df.groupby('구매자연령')['구매금액'].sum().sort_values(ascending=False)
age_sales.plot(figsize=(10, 3), kind='bar', rot=0)

## RFM 분석

RFM 분석을 통해 고객을 최근성(Recency), 빈도(Frequency), 금액(Monetary) 기준으로 세분화합니다. RFM 분석은 고객의 가치를 평가하고 마케팅 전략을 수립하는 데 유용한 도구입니다.


* RFM 분석을 위해서는 고객ID 가 있어야 하지만, 여기에서는 고객 ID를 알 수 없습니다.
* 고객 식별이 어려운 경우, 고객 집단이나 상품을 중심으로 분석하여 세그먼트를 정의합니다.
* 분석 결과를 활용해 그룹별로 프로모션 전략, 채널 운영, 상품 추천 등을 개선할 수 있습니다.

In [ ]:
# 최근성(Recency): 가장 최근 구매일
df['구매일'] = pd.to_datetime(df['구매주차일자'].str.split('~').str[0].str.strip())
recency = df.groupby('구매자연령')['구매일'].max()

# 빈도(Frequency): 구매 빈도
frequency = df.groupby('구매자연령')['테이블순번'].count()

# 금액(Monetary): 총 구매 금액
monetary = df.groupby('구매자연령')['구매금액'].sum()

rfm = pd.concat([recency, frequency, monetary], axis=1)
rfm.columns = ['Recency', 'Frequency', 'Monetary']
rfm

In [ ]:
recency = df.groupby('대분류')['구매일'].max()

# 빈도(Frequency): 구매 빈도
frequency = df.groupby('대분류')['테이블순번'].count()

# 금액(Monetary): 총 구매 금액
monetary = df.groupby('대분류')['구매금액'].sum()

rfm = pd.concat([recency, frequency, monetary], axis=1)
rfm.columns = ['Recency', 'Frequency', 'Monetary']
rfm

### plotly
* 파이 차트: 제품 카테고리별 매출 비중을 파이 차트로 표현하여 전체적인 매출 구조를 파악합니다.
* Treemap, Sunburst Chart: 제품 카테고리별 매출 비중을 시각적으로 표현하여 전체적인 매출 구조를 파악합니다.
* 히스토그램: 연령대별, 제품 카테고리별, 구매 주차별 구매 금액 및 수량 분포를 확인하여 데이터의 분포를 시각적으로 파악합니다.

In [ ]:
# 제품 카테고리별 판매 성과
px.pie(
    df,
    names='대분류',
    values='구매금액',
    width=400, height=400,
    title='제품 카테고리별 판매 성과'
)

In [ ]:
# Treemap Chart 시각화
px.treemap(
    df,
    path=['대분류', '중분류', '소분류'],  # 계층 구조를 나타내는 열
    values='구매금액',  # 크기를 나타내는 열
    width=900, height=600,
    title='제품 카테고리별 구매 금액 Treemap Chart'
)

In [ ]:
# Sunburst Chart 시각화
px.sunburst(
    df,
    path=['대분류', '중분류', '소분류'],  # 계층 구조를 나타내는 열
    values='구매금액',  # 크기를 나타내는 열
    width=900, height=900,
    title='제품 카테고리별 구매 금액 Sunburst Chart'
)

In [ ]:
import plotly.graph_objects as go

# 데이터 준비
category_counts = df['대분류'].value_counts()
subcategory_counts = df.groupby('중분류')['대분류'].count()
subcategory_grouped = df.groupby(['대분류', '중분류']).size()

# 노드 정의
main_categories = list(df['대분류'].unique())
sub_categories = list(df['중분류'].unique())
nodes = main_categories + sub_categories
node_labels = nodes

# 링크 정의
source_indices = []
target_indices = []
values = []

# 대분류 -> 중분류 흐름 데이터 구성
for main_category in main_categories:
    for sub_category in sub_categories:
        value = subcategory_grouped.get((main_category, sub_category), 0)
        if value > 0:
            source_indices.append(main_categories.index(main_category))
            target_indices.append(len(main_categories) + sub_categories.index(sub_category))
            values.append(value)

# Sankey 그래프 정의
fig = go.Figure(go.Sankey(
    node=dict(
        pad=15,  # 노드 간 간격
        thickness=20,  # 노드 두께
        line=dict(color="black", width=0.5),  # 노드 테두리
        label=node_labels,  # 노드 이름
    ),
    link=dict(
        source=source_indices,  # 출발 노드
        target=target_indices,  # 도착 노드
        value=values,  # 링크 값
    )
))

# 레이아웃 설정
fig.update_layout(
    title_text="대분류와 중분류 간 데이터 흐름 (Sankey Diagram)",
    font_size=12,
    width=800,
    height=600,
)

# 그래프 출력
fig.show()


In [ ]:
df.groupby('구매자연령')['구매금액'].describe().style.background_gradient().format('{:,.0f}')

In [ ]:
# 연령대별 구매 금액 히스토그램
px.histogram(df, x='구매금액', color='구매자연령', nbins=100, marginal="violin",
             title='연령대별 구매 금액 분포')

In [ ]:
df.groupby('대분류')['구매금액'].describe().style.background_gradient().format('{:,.0f}')

In [ ]:
# 대분류별 구매 금액 히스토그램
px.histogram(df, x='구매금액', color='대분류', nbins=100, marginal="violin",
             title='제품 대분류별 구매 금액 분포')

In [ ]:
# 구매 주차별 구매 수량 히스토그램
px.histogram(df, x='구매주차', y='구매수량', color='구매주차', title='구매 주차별 구매 수량', histfunc='sum')

## 군집화

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.cluster import KMeans

# 데이터 전처리
# 필요한 컬럼만 선택
columns_to_use = ['구매주차', '구매자연령', '대분류', '중분류', '구매수량', '구매금액']
df_selected = df[columns_to_use].copy()

# 범주형 데이터를 Label Encoding
label_encoders = {}
for col in ['구매주차', '구매자연령', '대분류', '중분류']:
    le = LabelEncoder()
    df_selected[col] = le.fit_transform(df_selected[col])
    label_encoders[col] = le

# 데이터 스케일링
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_selected)

# 최적의 클러스터 수 결정 (Elbow Method)
inertia = []
for k in range(1, 51):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(df_scaled)
    inertia.append(kmeans.inertia_)

# Elbow Method 시각화
plt.figure(figsize=(8, 5))
plt.plot(range(1, 51), inertia, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.title('Elbow Method')
plt.show()

In [ ]:
# K-Means Clustering
optimal_clusters = 5  # Elbow Method로 최적 클러스터 수를 선택하지만 여기에서는 임의로 지정
kmeans = KMeans(n_clusters=optimal_clusters, random_state=42)

In [ ]:
df['cluster'] = kmeans.fit_predict(df_scaled)
df

In [ ]:
# 군집별 수량 확인
cluster_counts = df['cluster'].value_counts().reset_index()
cluster_counts.columns = ['Cluster', 'Count']
print("Cluster Counts:")
print(cluster_counts)

In [ ]:
# 군집별 주요 통계 분석
# 구매금액, 구매수량, 구매자연령 등 주요 변수에 대해 평균/최대값 계산
cluster_summary = df.groupby('cluster').agg({
    '구매수량': ['mean', 'sum'],
    '구매금액': ['mean', 'sum', 'max'],
    '구매자연령': lambda x: x.mode()[0],  # 최빈값
    '대분류': lambda x: x.mode()[0],     # 최빈값
    '중분류': lambda x: x.mode()[0]      # 최빈값
}).reset_index()

# 컬럼 이름 정리
cluster_summary.columns = ['Cluster', 
                           'Avg Quantity', 'Total Quantity', 
                           'Avg Amount', 'Total Amount', 'Max Amount', 
                           'Most Common Age', 'Most Common Category', 'Most Common Subcategory']
print("\nCluster Summary:")
cluster_summary.style.background_gradient()

In [ ]:
pd.crosstab(df['대분류'], df['cluster']).style.background_gradient()

In [ ]:
df[(df['대분류'] == '반려동물용품') & (df['cluster'] == 3)]